In [83]:
using OrdinaryDiffEq, Flux, Optim, Random, Plots
using Zygote
using ForwardDiff
using LinearAlgebra, Statistics
using ProgressBars, Printf
using Flux.Optimise: update!, ExpDecay
using Flux.Losses: mae, mse
using BSON: @save, @load
using DiffEqBase

In [87]:
Random.seed!(1234);

In [105]:
# Входные параметры

is_restart = false;
p_cutoff = 0.0;                                                     # Непонятно для чего
n_epoch = 1000;                                                     # Количество эпох. Максимальное
n_plot = 100;                                                       # Частота формирования графиков. Через сколько эпох
opt = ADAMW(0.001, (0.9, 0.999), 1.f-8);                            # Оптимизатор
datasize = 10;                                                     # Размер датасетов?
tstep = 1;                                                          # Шаг времени для татасетов? или типа их количество?
n_exp_train = 6;                                                   # Размер данных для обучения
n_exp_test = 2;                                                    # Размер даных для теста
n_exp = n_exp_train + n_exp_test;                                   # Общий размер данных
noise = 5.f-2;                                                      # ШУМ
ns = 5;                                                             # Количество веществ
nr = 4;                                                             # Количество хим. реакций
k = Float32[0.1, 0.2, 0.13, 0.3];                                   # константы хим. реакций
alg = Tsit5();                                                      # Алгоритм для решения ОДУ?
atol = 1e-5;                                                        # Параметр точности для ОДУ
rtol = 1e-2;                                                        # Параметр точности для ОДУ

maxiters = 10000;                                                   # Не понял для чего нужно

lb = 1.f-5;
ub = 1.f1;

In [89]:
# Generate data sets
u0_list = rand(Float32, (n_exp, ns));
u0_list[:, 1:2] .+= 2.f-1;
u0_list[:, 3:end] .= 0.f0;

# В результате u0_list - матрица ns столбцов и n_exp строк, в которой 3+ столбцы нули
tspan = Float32[0.0, datasize * tstep];
tsteps = range(tspan[1], tspan[2], length=datasize);
ode_data_list = zeros(Float32, (n_exp, ns, datasize));
std_list = [];

* u0_list       - матрциа размерности n_exp * ns
* tspan         - список из двух элементов, где первый элемент = 0, это начало времени, а второй = datasize * tstep, коненц времени
* tsteps        - что-то типа списка с границами tspan, и некоторым шагом
* ode_data_list - datasize матриц размерности n_exp * ns, заполненные 0

In [90]:
function max_min(ode_data)
    return maximum(ode_data, dims=2) .- minimum(ode_data, dims=2) .+ lb
end

# Выдает разницу между максимальным и минимальным значением в ode_data

max_min (generic function with 1 method)

In [91]:
function trueODEfunc(dydt, y, k, t)
    dydt[1] = -2 * k[1] * y[1]^2 - k[2] * y[1];
    dydt[2] = k[1] * y[1]^2 - k[4] * y[2] * y[4];
    dydt[3] = k[2] * y[1] - k[3] * y[3];
    dydt[4] = k[3] * y[3] - k[4] * y[2] * y[4];
    dydt[5] = k[4] * y[2] * y[4];
end

# Это процедура, которая описывает, как изменяются переменные в системе ОДУ по времени.

trueODEfunc (generic function with 1 method)

In [92]:
# Дале идет цикл с решением ODE для каждой строки матрицы u0_list.
# Решение записывается в std_list

for i in 1:n_exp
    u0 = u0_list[i, :];                                          # Берется i-я строка из u0_list начальных условий
    prob_trueode = ODEProblem(trueODEfunc, u0, tspan, k);        # ODEProblem не вызывает trueODEfunc напрямую, 
    # она используется для создания объекта задачи ODEProblem, который потом решается с помощью solve()
    ode_data = Array(solve(prob_trueode, alg, saveat=tsteps));   # Матрица решений ОДУ, alg - алгоритм решения ОДУ
    ode_data += randn(size(ode_data)) .* ode_data .* noise       # Шумы из N(0,1)
    ode_data_list[i, :, :] = ode_data                            # тут шумы добавляются в матрицу
    push!(std_list, max_min(ode_data));
end

y_std = maximum(hcat(std_list...), dims=2);                      # Вектор наибольшей разницы 

Более подробное разъяснение строки 

``` prob_trueode = ODEProblem(trueODEfunc, u0, tspan, k); ```

* prob_trueode = ODEProblem(trueODEfunc, u0, tspan, k);
* Создается задача для решения системы ОДУ, где 
* trueODEfunc  - функция, описывающая систему ОДУ
* u0           - начальные условия 
* tspan        - врменной интервал 
* k            - параметры системы, которые могут влиять на поведение ОДУ
* prob_trueode - это объект, описывающий задачу для последующего решения ОДУ

In [93]:
u0 = u0_list[1, :]
p = randn(Float32, nr * (ns + 1)) .* 1.f-1;             # Рандомный вектор длины nr * (ns + 1)

In [94]:
b0 = -10.0 

function p2vec(p)
    w_b = p[1:nr] .+ b0;                        # вектор смещений
    w_out = reshape(p[nr + 1:end], ns, nr);     # матрица будущих весов нейронной сети??
    # w_out = clamp.(w_out, -2.5, 2.5);
    w_in = clamp.(-w_out, 0, 2.5);              # Конвертируют w_out, и все числа к диапазону [0, 2.5]
    return w_in, w_b, w_out
end

p2vec (generic function with 1 method)

Функция ```p2vec(p)``` берет вектор параметров p, и преобразует в три компоненты: входные веса, смещения и выходные веса

In [95]:
function crnn!(du, u, p, t)
    w_in_x = w_in' * @. log(clamp(u, lb, ub));
    du .= w_out * @. exp(w_in_x + w_b);
end

crnn! (generic function with 1 method)

`w_in'` — транспонированная матрица <br>
`clamp(u, lb, ub)` — ограничивает значение вектора `u` в пределах от `lb` до `ub`<br>
`log(clamp(u, lb, ub))` — поэлементное взятие логарифма от ограниченных значений <br>
`@.` — макрос, который автоматически применяет все операции поэлементно (векторизует их), сокращая необходимость писать `.`, например, в `log.` <br>
`du .= ...` — обновление вектора du поэлементно новыми значениями <br>
<br>
<br>
функция `crnn!` обновляет вектор `du`

In [96]:
prob = ODEProblem(crnn!, u0, tspan, saveat=tsteps, atol=atol, rtol=rtol)

ODEProblem with uType Vector{Float32} and tType Float32. In-place: true
timespan: (0.0f0, 10.0f0)
u0: 5-element Vector{Float32}:
 0.21077032
 0.4572782
 0.0
 0.0
 0.0

`prob = ODEProblem(...)` создает объект задачи для решения системы ОДУ. Этот объект включает в себя: <br>
* Определение функции ОДУ (`crnn!`), которая описывает изменение переменных во времени. <br>
* Начальные условия (`u0`), задающие начальное состояние системы. <br>
* Интервал времени для решения (`tspan`). <br>
* Параметры точности решения (абсолютная и относительная ошибки). <br>
* Временные точки, в которых необходимо сохранить решение (`saveat=tsteps`). <br>
Решение задачи: После создания объекта `prob` его можно передать в численный решатель, например, с помощью функции `solve`, чтобы найти решение системы дифференциальных уравнений. Решение будет возвращено в виде набора значений переменных в заданные моменты времени. <br>
<br>
После этого нужно вызывать `solve()`

In [97]:
function predict_neuralode(u0, p)
    global w_in, w_b, w_out = p2vec(p);
    pred = clamp.(Array(solve(prob, alg, u0=u0, p=p;
                  maxiters=maxiters)), -ub, ub)
    return pred
end

predict_neuralode (generic function with 1 method)

Здесь используется ключевое слово `global`, потому что `w_in`, `w_b`, и `w_out` объявлены как глобальные переменные, и их нужно обновить для последующих вычислений. <br>
`pred = clamp.(Array(solve(prob, alg, u0=u0, p=p; maxiters=maxiters)), -ub, ub)` - Это основная строка, в которой происходит решение системы дифференциальных уравнений и получение предсказания. <br>
`clamp.` — это поэлементная операция, которая ограничивает значения предсказаний в пределах от `-ub` до `ub`.
`maxiters` - макисмальное число операций, которые решатель может выполнить. <br>
<br>
<br>
Эта функция используется для предсказания на основе нейронной ODE-модели, которая основана на дифференциальных уравнениях.

In [98]:
function display_p(p)
    w_in, w_b, w_out = p2vec(p);
    println("species (column) reaction (row)")
    println()
    println("w_in")
    show(stdout, "text/plain", round.(w_in', digits=3))

    println("\nw_b")
    show(stdout, "text/plain", round.(exp.(w_b'), digits=3))

    println("\nw_out")
    show(stdout, "text/plain", round.(w_out', digits=3))
    println("\n\n")
end
display_p(p)

species (column) reaction (row)

w_in
4×5 Matrix{Float64}:
 0.037  0.159  0.0    0.0  0.0
 0.156  0.068  0.031  0.0  0.0
 0.0    0.047  0.005  0.0  0.0
 0.0    0.0    0.0    0.0  0.0
w_b
1×4 Matrix{Float64}:
 0.0  0.0  0.0  0.0
w_out
4×5 Matrix{Float32}:
 -0.037  -0.159   0.044  0.073  0.17
 -0.156  -0.068  -0.031  0.104  0.064
  0.074  -0.047  -0.005  0.019  0.019
  0.077   0.059   0.048  0.042  0.002




Итого: функция `display_p` превращает вектор параметров `p` в три компонента: входные веса `w_in`, смещения `w_b` и выходные веса `w_out`

In [99]:
function loss_neuralode(p, i_exp)
    pred = predict_neuralode(u0_list[i_exp, :], p)
    loss = mae(ode_data_list[i_exp, :, :] ./ y_std, pred ./ y_std)
    return loss
end

loss_neuralode (generic function with 1 method)

Функция принимает два аргумента:
* `p` — вектор параметров нейронной сети (веса и смещения), которые используются для вычисления предсказаний.
* `i_exp` — индекс эксперимента, который указывает на конкретный набор данных, с которым будет сравниваться предсказание.

`predict_neuralode` - функция, записанная ранее, она возвращает массив педсказанных значений
`loss = mae(ode_data_list[i_exp, :, :] ./ y_std, pred ./ y_std)` - вычисляется ошибка между реальными данными `ode_data_list[i_exp, :, :]` и предсказаниями `pred` <br>
<br>
`y_std` — это стандартное отклонение (нормировочный коэффициент), которое используется для масштабирования как реальных данных, так и предсказаний. <br>
`./ y_std` — операция поэлементного деления на стандартное отклонение для нормализации данных (как реальных, так и предсказанных значений). <br>
`mae` — это функция для вычисления среднеабсолютной ошибки (Mean Absolute Error), которая измеряет среднее по всем значениям разницу между предсказанными и реальными данными после нормализации. <br>
<br>
<br>
В результате функция вычисляет и возвращает значение ошибки (разницу по mae между реалдьными и предсказанными данными) для конкретного эксперимента

In [100]:
species = ["A", "B", "C", "D", "E"];
cbi = function (p, i_exp)
    ode_data = ode_data_list[i_exp, :, :]
    pred = predict_neuralode(u0_list[i_exp, :], p)
    list_plt = []
    for i in 1:ns                                                      # Цикл по всем участвующим веществам
        plt = scatter(tsteps, ode_data[i,:], 
                      markercolor=:transparent,                        # Цвет маркеров
                      label="Exp",                                     # Подпись для легенды
                      framestyle=:box)                                 # Стиль рамки графика
        plot!(plt, tsteps, pred[i,:], label="CRNN-ODE")                # Добавление на тот же график значений pred[i, :] для i-го компонента
        plot!(xlabel="Time", ylabel="Concentration of " * species[i])  # Установка подписей для осей графика:

        # Какая-то тема с легендами, не важно
        if i==1
            plot!(plt, legend=true, framealpha=0)
        else
            plot!(plt, legend=false)
        end

        push!(list_plt, plt)                                           # list_plt содержит в себе все графики
    end
    plt_all = plot(list_plt...)                                        # объединение всех графиков list_plt в один большой график

    png(plt_all, string(joinpath(@__DIR__, "figs", ""), i_exp))        # сохранение объединения графиков в папке в виде .png                                            # Здесь сохраняется png файл
    # joinpath самостоятельно определяет относительный путь к папке/файлу
    
    return false
end

#15 (generic function with 1 method)

* `ode_data = ode_data_list[i_exp, :, :]` - Получение реальных данных для эксперимента с индексом `i_exp` из массива `ode_data_list`, который содержит результаты симуляций (реальные данные). Эти данные будут использоваться для построения графиков. <br>
* `pred` — предсказания нейронной сети, которые будут отображены на графике для сравнения с реальными данными.
* `plt = scatter(tsteps, ode_data[i,:], markercolor=:transparent, label="Exp", framestyle=:box)` - 
* `ode_data[i,:]` - реальные данные для компонента i (по оси Y) <br>
<br>
<br>
Функция cbi визуализирует результаты экспериментов, сравнивая реальные данные с предсказаниями модели CRNN-ODE. Она создает и сохраняет графики для каждого компонента, выводя на экран данные по концентрациям компонентов с течением времени.

In [101]:
list_loss_train = []
list_loss_val = []
iter = 1

1

In [102]:
cb = function (p, loss_train, loss_val)
    global list_loss_train, list_loss_val, iter          # Чтобы изменения этих переменных сохранялись глобально
    push!(list_loss_train, loss_train)                   # добавляет значение ошибки на трейне `loss_train` в глобальный список `list_loss_train`
    push!(list_loss_val, loss_val)                       # добавляет значение ошибки на валид. выборке `loss_val` в глобальный список `list_loss_val`

    if iter % n_plot == 0    
        display_p(p)

        @printf("min loss train: %.4e, and validation: %.4e\n", minimum(list_loss_train), minimum(list_loss_val))
        # Выводит минимальные значения ошибок на train and validat наборах данных в формате %.4e - експоненциальная форма

        list_exp = randperm(n_exp)[1:1];
        println("update plot for ", list_exp)
        for i_exp in list_exp
            cbi(p, i_exp)                                                     # строит и сохраняет графики
        end

        plt_loss = plot(list_loss_train, xscale=:log10, yscale=:log10, label="train");     # создает график ошибок на train наборе
        plot!(plt_loss, list_loss_val, label="val");                          # добавляет к этому же графику кривую на validat наборе


        png(plt_loss, joinpath(@__DIR__, "figs", "loss.png"));                # тут сохраняюется loss файл

        @save joinpath(@__DIR__, "checkpoint", "mymodel.bson") p opt list_loss_train list_loss_val iter                 # тут сохраняется bson файл
    end

    iter += 1;
end

#17 (generic function with 1 method)

* `loss_train` — значение ошибки на тренировочном наборе.
* `loss_val` — значение ошибки на валидационном наборе.

`if iter % n_plot == 0 ` запускается, если текущая итерация `iter` делится на n_plot без остатка. Это частота формирвоания графиков <br><br>
`list_exp = randperm(n_exp)[1:1];` генерирует одно "случайное" число от `1` до `n_exp-1` <br><br>
`@save joinpath(@__DIR__, "checkpoint", "mymodel.bson") p opt list_loss_train list_loss_val iter` сохраняет текущие параметры модели `p`, оптимизатор `opt`, списки ошибок `list_loss_train`, и `list_loss_val`, а также текущую итерацию `iter` в файл `mymodel.bson`

In [106]:
if is_restart
    #@load "./checkpoint/mymodel.bson" p opt list_loss_train list_loss_val iter;                                        Переписываю путь
    @load joinpath(@__DIR__, "checkpoint", "mymodel.bson") p opt list_loss_train list_loss_val iter;                         # тут читается bson файл
    iter += 1; 
end

In [107]:
i_exp = 1
epochs = ProgressBar(iter:n_epoch)                  
loss_epoch = zeros(Float32, n_exp);
for epoch in epochs                                                # Основной цикл по эпохам
    global p
    for i_exp in randperm(n_exp_train) 

        grad = gradient(p) do x
            Zygote.forwarddiff(x) do x
                loss_neuralode(x, i_exp)
            end
        end
        update!(opt, p, grad[1])                                   # Изменяет параметр p по оптимизатору opt в соответствии с градиентом
    end

    for i_exp in 1:n_exp                                           # После обновления параметров вычисляется значение ошибки для каждого эксперимента
        loss_epoch[i_exp] = loss_neuralode(p, i_exp)
    end

    loss_train = mean(loss_epoch[1:n_exp_train]);                  # средняя ошибка для тестового набора
    loss_val = mean(loss_epoch[n_exp_train + 1:end]);              # средняя ошибка для валидационного набора
    set_description(epochs, string(@sprintf("Loss train %.4e val %.4e", loss_train, loss_val)))
    cb(p, loss_train, loss_val);
end

0.0%┣                                           ┫ 0/1.0k [00:01<-22:-19, -1s/it]
Loss train 2.7984e-01 val 3.5451e-01 0.1%┣    ┫ 1/1.0k [00:29<Inf:Inf, InfGs/it]
Loss train 2.7955e-01 val 3.5422e-01 1.8%┣▏       ┫ 18/1.0k [00:29<27:55, 2s/it]
Loss train 2.7915e-01 val 3.5381e-01 3.5%┣▎       ┫ 35/1.0k [00:29<13:45, 1it/s]
Loss train 2.7852e-01 val 3.5318e-01 5.3%┣▍       ┫ 53/1.0k [00:29<08:51, 2it/s]
Loss train 2.7749e-01 val 3.5213e-01 7.3%┣▋       ┫ 73/1.0k [00:29<06:16, 2it/s]
Loss train 2.7602e-01 val 3.5064e-01 9.2%┣▊       ┫ 92/1.0k [00:29<04:52, 3it/s]


species (column) reaction (row)

w_in
4×5 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
w_b
1×4 Matrix{Float64}:
 0.0  0.0  0.0  0.0
w_out
4×5 Matrix{Float32}:
 0.003  0.216  0.787  0.818  0.912
 0.001  0.296  0.542  0.868  0.825
 0.001  0.309  0.729  0.76   0.758
 0.0    0.409  0.779  0.775  0.732


min loss train 2.7522e-01 val 3.4982e-01
update plot for [1]


SystemError: SystemError: opening file "c:\\Users\\GeraschenkoKM\\Desktop\\Petrochemistry\\My work CRNN\\Chemistry\\MyTest\\figs\\1.png": No such file or directory

In [81]:
for i_exp in randperm(n_exp_train)
    println(i_exp)
end

1
4
2
5
6
3


`epochs = ProgressBar(iter:n_epoch)` создает индикатор прогресса эпох от iter до n_epoch <br>
В `loss_epoch` будет храниться значение ошибки для каждого эксперимента <br><br>
`for i_exp in randperm(n_exp_train)` начало цикла по перемешанному вектору значений от 1 до `n_exp_train` <br><br>
`set_description(epochs, string(@sprintf("Loss train %.4e val %.4e", loss_train, loss_val)))` нужно для динамического обновления прогресс-бара значениями потерь на обучающей и валидационной выборках, чтобы можно было отслеживать их изменения в реальном времени.